## Introduction ##

In this project, we are working for a mining company to find the best place for a new oil well. Given a set of data, we will build a linear regression model that will predict the volume of oil reserves for new wells. Our goal is to select the region with the highest profit margin based on the predicted volumes. We will evaluate and justify our selection by calculating the average profit, 95% condience interval and risk of losses (profit loss probability expressed as a percentage) using the Bootstrapping technique. The data we are provided are as follows:

**geo_data_0.csv, geo_data_1.csv, geo_data_2.csv**  
- *id*: unique oil well identifier
- *f0, f1, f2*: three features of points (their specific meaning is unimportant, but the features themselves are significant)
- *product*: volume of reserves in the oil well (thousand barrels)

For calculating potential profitability and risk, we are provided the following development conditions:
- A study of 500 points is carried with picking the best 200 points for the profit calculation.
- The budget for development of 200 oil wells is 100 USD million.
- One barrel of raw materials brings 4.5 USD of revenue. The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).
- Only regions with the risk of losses lower than 2.5% will be considered. From the ones that fit the criteria, the region with the highest average profit should be selected.

## Imports ##

**Packages, Libraries, and Modules**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st

## Reading and Inspecting the Data ##

**Data info and Sample**

In [2]:
# When cloning this project, ensure directory/file paths are correct with respect to user's operating system
region_0 = pd.read_csv('datasets/geo_data_0.csv')
region_1 = pd.read_csv('datasets/geo_data_1.csv')
region_2 = pd.read_csv('datasets/geo_data_2.csv')

region_0.info()
display(region_0.sample(10))

region_1.info()
display(region_1.sample(10))

region_2.info()
display(region_2.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
20823,2DHY0,1.937329,0.331056,2.001616,28.899223
51379,mjW5x,-0.842372,0.573771,3.425880,54.528586
9374,J7be7,1.759669,-0.098984,3.124650,104.348377
59756,NkI0f,-0.957420,0.471977,-4.172174,25.544510
67890,d2qQp,-0.163030,1.097857,-1.594005,7.745261
37171,L4ukK,1.598888,-0.124044,0.289769,58.523787
25214,HJugI,0.976285,-0.489327,7.923635,147.873573
63814,DxmMn,1.638379,-0.472472,-2.454135,86.908642
66766,fqswp,0.173234,0.942078,-1.636512,39.023846
66171,eCWpN,-0.924806,0.035794,5.395557,104.397843


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
93171,49Fqn,13.646007,5.451887,0.006049,0.000000
7438,cXIRN,8.737388,-5.613791,-0.003143,0.000000
55976,YyCvd,-6.500767,2.808456,4.008309,110.992147
91199,0mso8,-6.432763,-8.095575,-0.005905,3.179103
90036,q9ql2,3.672887,-11.661420,4.984688,134.766305
96025,u0T70,-9.117380,3.067011,4.002817,110.992147
64654,CpGw3,5.660266,3.574557,3.002154,80.859783
1787,qkXYK,11.935348,-10.335086,1.999363,53.906522
5055,K1pqo,2.656001,-5.735697,5.005218,134.766305
6889,x3qMU,8.053023,-13.407494,3.012910,80.859783


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
62718,RMDO5,-1.157487,-1.951781,1.726350,44.771841
22976,mqJnc,1.162697,-0.875760,-0.657080,30.179231
68095,PFWO6,0.814750,-0.075592,4.438996,106.881234
53276,TYXlh,0.629148,-3.177217,5.029556,179.105930
5688,HakxU,0.423200,0.822557,5.229675,86.223820
2083,FU4kX,0.786466,-1.850974,8.066554,116.575609
56043,O0Ai7,1.452218,1.041417,0.772754,25.800099
57858,nPLWm,0.538214,0.262635,2.351075,113.386961
80423,dpXXX,-0.991385,-1.186877,0.925543,121.908448
47934,P8Q3Y,-5.181205,3.613947,3.200127,98.238020


The datasets appear to be in good order. Each region (dataset) has 100,000 observations/entries, no missing values, and the datatypes are appropriate. Next, we'll ensure there are no duplicate values.

**Checking for Duplicate Values**

In [3]:
print('Number of duplicates for Region 0: ', region_0.duplicated().sum())
print('Number of duplicates for Region 1: ', region_1.duplicated().sum())
print('Number of duplicates for Region 2: ', region_2.duplicated().sum())

Number of duplicates for Region 0:  0
Number of duplicates for Region 1:  0
Number of duplicates for Region 2:  0


There are no duplicate rows in any of the datasets.

## Model Training and Predictions ##

For each region, we'll split the data into a training set and validation set with a 3:1 ratio:
- *training*: 75%
- *validation*: 25%

A linear regression model will be trained on the features (i.e. f0, f1, f2) for each region with a target variable of product volume. We will store the predictions and targets for each validation set in a list, which will be used later on for profit and risk calculations.

In [4]:
region_list = [region_0, region_1, region_2]
predict_list = []
target_list = []
model = LinearRegression()

for region in region_list:
    features = region[['f0','f1', 'f2']]
    target = region['product']

    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    predict_list.append(pd.Series(predicted, index=target_valid.index))
    target_list.append(target_valid)

To faciliate displaying the different calculations during our analysis, we'll define a function,  **compile_df( )**, as follows:

**Parameters**:
- *data_list*: List object containing the calculated values for each region.
- *column_names*: Column names for the data.

**Returns**: DataFrame object compiled from the parameters.

In [5]:
def compile_df(data_list, column_names):
    table = [['Region 0'], ['Region 1'], ['Region 2']]
    values= []
    
    for i in range(len(table)):
        values.append(table[i]+data_list[i])
    
    return pd.DataFrame(data=values, columns=column_names)

**Average Estimated Volume and RMSE**

In [6]:
cols = ['Region', 'Average Estimated Volume', 'RMSE']
data = []

for i in range(len(region_list)):
    data.append([predict_list[i].mean()] + [mean_squared_error(target_list[i], predict_list[i])**0.5])

display(compile_df(data, cols))

,Region,Average Estimated Volume,RMSE
0,Region 0,92.592568,37.579422
1,Region 1,68.728547,0.893099
2,Region 2,94.965046,40.029709


Our model predicts the average volume of reserves for region 0 and region 2 are about 92-95 product units (thousands of barrels), whereas region 1 is estimated to be about 68 product units. The RMSE suggests the model estimated values close to the true volume for region 1, whereas the other regions could potentially have large margins of errors.

## Preparing Profit Calculation ##

**Key Values**

The following key values used in calculating profits are stored in separate variables to facilitate future scalability and/or modularity:
- *revenue_per_unit*: 4,500 USD per unit of product
- *min_survey_size*: 500 points surveyed per region
- *wells_per_survey*: For each survey, top 200 points selected for profit calculation
- *budget_per_survey*: 100 million USD development budget for 200 wells

In [7]:
revenue_per_unit = 4500
min_survey_size = 500
wells_per_survey = 200
budget_per_survey = 100e+06 # 100 million expressed in scientific notation for readability

**Minimum Reserve Volume**

Given our conditions and budget, we can calculate the minimum oil reserve volume required for development without a loss.

In [8]:
print('Minimum reserve volume (thousands of barrels) per 200 wells: ', budget_per_survey/revenue_per_unit)
print('Minimum reserve volume (thousands of barrels) per well: ', (budget_per_survey/revenue_per_unit)/wells_per_survey, '\n')

for i in range(len(region_list)):
    print('Region ', i, ' average volume: ', region_list[i]['product'].mean())

Minimum reserve volume (thousands of barrels) per 200 wells:  22222.222222222223
Minimum reserve volume (thousands of barrels) per well:  111.11111111111111 

Region  0  average volume:  92.50000000000001
Region  1  average volume:  68.82500000000002
Region  2  average volume:  95.00000000000004


We can see the average estimated volume is fairly close to the actual average volume of reserves for each region. Based on our budget and development conditions, we would need approximately 22,222 product units to develop 200 wells above a loss (approximately 111 product units per well). The average for each region is below the the profit threshold. This indicates when we consider the top 200 wells per region, we'll need to ensure we have a large enough volume to compensate for any wells below the minimum/average.

## Calculating Profit and Risk ##

To calculate profits based on our predicted volumes, we'll define the function, **calc_profit_avg_volume( )**, as follows:

**Parameters**:
- *target*: Series object of true target values
- *prediction*: Series object of predicted values

**Returns**: A tuple of the profit for 200 new wells and averge volume of the selected wells.
- The estimated volumes are sorted in descending order. The true target values, corresponding to the indices of the sorted predictions, are used for revenue and average volume calculations.
- Revenue less budget results in overall profit for top 200 wells by volume.

In [9]:
def calc_profit_avg_volume(target, prediction):
    prediction_sorted = prediction.sort_values(ascending=False)
    selected_wells = target[prediction_sorted.index].iloc[:wells_per_survey]
    avg_volume = selected_wells.mean()

    return ((revenue_per_unit*selected_wells.sum()) - budget_per_survey), avg_volume

In [10]:
cols = ['Region', 'Profit', 'Average Volume']

data = []
for i in range(len(region_list)):
    profit, avg_volume = calc_profit_avg_volume(target_list[i], predict_list[i])
    data.append([profit] + [avg_volume])

display(compile_df(data, cols))

,Region,Profit,Average Volume
0,Region 0,3.320826e+07,148.009178
1,Region 1,2.415087e+07,137.945408
2,Region 2,2.710350e+07,141.226111


According to our model predictions, region 0 appears to be the most profitable with a calculated profit of approximately 33.2 Million USD and average reserve volume of 148 product units. 

According to our model's predictions, each region should be profitable with average reserve volumes above the calculated minimums. While this appears true, it is not quite indicative of what kind of profit we can actually achieve as these values were calculated using overall top 200 wells in each region (by predictions). We only have the budget to make initial measurements at 500 randomly selected locations, it's highly unlikely that the overall top 200 wells are contained in such a small sample.

**Bootstrapping Samples**

Using the bootstrapping technique, we'll generate 1,000 samples per region to  find the distribution of profit. For the sampling parameters, we'll opt to not replace chosen values as to simulate selecting unique well points for our calculations. Additionally, we'll calculate the 95% confidence interval and risk of losses.

In [11]:
state = np.random.RandomState(12345)
cols = ['Region', 'Average Estimated Profit', '95% Confidence Interval', 'Risk of Loss']
data = []

for idx in range(len(region_list)):
    values = []
    
    for i in range(1000):
        predict_subsample = predict_list[idx].sample(n=min_survey_size, replace=False, random_state=state)
        target_subsample = target_list[idx][predict_subsample.index]
        
        revenue, avg_volume = calc_profit_avg_volume(target_subsample, predict_subsample)
        values.append(revenue)
        
    sample = pd.Series(values)
    confidence_interval = st.t.interval(0.95, len(sample)-1, sample.mean(), sample.sem())
    risk_of_loss = (sample[sample < 0].count()/sample.size)*100
    
    data.append([sample.mean()] + [confidence_interval] + [risk_of_loss])

display(compile_df(data, cols))

,Region,Average Estimated Profit,95% Confidence Interval,Risk of Loss
0,Region 0,3.807109e+06,"(3646247.0295118885, 3967970.784669607)",7.2
1,Region 1,4.547854e+06,"(4419792.413792762, 4675916.281521496)",1.3
2,Region 2,3.892171e+06,"(3728577.3201472466, 4055764.1535641546)",7.3


When looking over the predictions of 1,000 bootstrapped samples, our calculations suggest that the most reliable profits would be gained in region 1. While the previous findings showed that each region would be profitable and that region 0 appeared to be the highest, our calculations here show that region 1 has the higher distribution of profits (higher 95 confidence interval) as well as the lowest risk of loss percentage. Furthermore, region 1 is the only region is below our acceptable risk threshold of 2.5%.

## Conclusion ##

We saw two different approaches that had potentially conflicting results when using model predictions in the mining company's business venture. While the selected locations using only the model predictions showed potential profits were an order of magnitude higher in region 0, the bootstrapping technique showed there could be other factors that drive business decisions, such as the overall profit distribution and risks. As such, it is imperative to understand both short and long term goals when providing insight for operational guidance. While region 0 may appear to generate the most revenue, long term development could potentially result in lower than expected profits. If higher profit distribution with lower associated risk of loss is more important to operations, new wells should be developed in region 1.